In [115]:
import scipy
import librosa
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import scipy.io.wavfile
import time
import IPython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import json

In [2]:
gt = pd.read_csv('/content/drive/MyDrive/COSRMAL_CHALLENGE/train.csv')
gt.head()

,id,container id,scenario,background,illumination,width at the top,width at the bottom,height,depth,container capacity,container mass,filling type,filling level,filling density,filling mass,object mass,handover starting frame,handover start timestamp,handover hand,action,nframes,folder_num,file_name,num,subject,filling_type,filling_level,back,light,camera_id,start,end
0,0,2,2,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,2,1,0.82,76.0,78.0,-1,-1,-1,1.0,291576,2,s2_fi2_fu1_b1_l0,70,2,2,1,1,0,2,0.75,3.5
1,1,7,0,0,0,193.0,193.0,241.0,69.0,3209.397,59.0,0,0,0.00,0.0,59.0,-1,-1,-1,0.0,118483,7,s0_fi0_fu0_b0_l0,0,0,0,0,0,0,2,-1.00,-1.0
2,2,2,0,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,3,1,1.00,93.0,95.0,-1,-1,-1,1.0,572008,2,s0_fi3_fu1_b1_l0,22,0,3,1,1,0,2,3.40,6.5
3,3,8,0,1,0,135.0,135.0,164.0,56.0,1239.840,31.0,0,0,0.00,0.0,31.0,-1,-1,-1,0.0,141680,8,s0_fi0_fu0_b1_l0,2,0,0,0,1,0,2,-1.00,-1.0
4,4,4,1,1,0,88.0,56.0,91.0,-1.0,296.000,86.0,1,1,0.34,45.0,131.0,-1,-1,-1,1.0,138681,4,s1_fi1_fu1_b1_l0,34,1,1,1,1,0,2,0.75,1.8


In [3]:
class AudioProcessing():
    def __init__(self,sample_rate,signal,frame_length_t=0.025,frame_stride_t=0.01,nfilt =64):
        
        self.sample_rate=sample_rate
        self.signal = signal
        self.frame_length_t=frame_length_t
        self.frame_stride_t=frame_stride_t
        self.signal_length_t=float(signal.shape[0]/sample_rate)
        self.frame_length=int(round(frame_length_t * sample_rate)) #number of samples
        self.frame_step=int(round(frame_stride_t * sample_rate))
        self.signal_length = signal.shape[0]
        self.nfilt=nfilt
        self.num_frames = int(np.ceil(float(np.abs(self.signal_length - self.frame_length)) / self.frame_step))
        self.pad_signal_length=self.num_frames * self.frame_step + self.frame_length
        self.NFFT=512
        
    def cal_frames(self):
        z = np.zeros([self.pad_signal_length - self.signal_length,8])
        pad_signal = np.concatenate([self.signal, z], 0)
        indices = np.tile(np.arange(0, self.frame_length), (self.num_frames, 1)) + np.tile(np.arange(0, self.num_frames * self.frame_step, self.frame_step), (self.frame_length, 1)).T
        frames = pad_signal[indices.astype(np.int32, copy=False)]
        return frames
        
    def calc_MFCC(self):
        # 291576
        pre_emphasis=0.97

        # (n,8)
        emphasized_signal=np.concatenate([self.signal[0,:].reshape([1,-1]),  self.signal[1:,:] - pre_emphasis * self.signal[:-1,:]], 0)
        z = np.zeros([self.pad_signal_length - self.signal_length,8])
        pad_signal = np.concatenate([emphasized_signal, z], 0)
        indices = np.tile(np.arange(0, self.frame_length), (self.num_frames, 1)) + np.tile(np.arange(0, self.num_frames * self.frame_step, self.frame_step), (self.frame_length, 1)).T
        frames = pad_signal[indices.astype(np.int32, copy=False)]
        frames=frames*np.hamming(self.frame_length).reshape(1,-1,1)
        frames=frames.transpose(0,2,1)
        mag_frames = np.absolute(np.fft.rfft(frames,self.NFFT))
        pow_frames = ((1.0 / self.NFFT) * ((mag_frames) ** 2))
        filter_banks = np.dot(pow_frames, self.cal_fbank().T)
        filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
        filter_banks = 20 * np.log10(filter_banks)  # dB
        filter_banks =filter_banks.transpose(0,2,1)
        
        return filter_banks
           
    def cal_fbank(self):
        
        low_freq_mel = 0
        high_freq_mel = (2595 * np.log10(1 + (self.sample_rate / 2) / 700))  
        mel_points = np.linspace(low_freq_mel, high_freq_mel, self.nfilt + 2)  
        hz_points = (700 * (10**(mel_points / 2595) - 1)) 
        bin = np.floor((self.NFFT + 1) * hz_points / self.sample_rate)
        fbank = np.zeros((self.nfilt, int(np.floor(self.NFFT / 2 + 1))))
        for m in range(1, self.nfilt + 1):
            f_m_minus = int(bin[m - 1])   # left
            f_m = int(bin[m])             # center
            f_m_plus = int(bin[m + 1])    # right

            for k in range(f_m_minus, f_m):
                fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
            for k in range(f_m, f_m_plus):
                fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
        return fbank

In [4]:
base_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/'
audio_folder = '/content/drive/MyDrive/COSRMAL_CHALLENGE/train/audio/'
os.makedirs(os.path.join(base_path, 'audios'), exist_ok=True)
mfcc_path = (os.path.join(base_path, 'audios', 'mfcc'))
raw_path = (os.path.join(base_path, 'audios', 'raw'))
# os.makedirs(mfcc_path,exist_ok=True)
# os.makedirs(raw_path,exist_ok=True)

# audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
# save_size=64
# ratio_step = 0.25
# count = 0
# pouring_or_shaking_list = []
# file_idx_list = []
# filling_type_list = []
# pbar = tqdm(total=len(audio_paths))

# for i, path in enumerate(audio_paths):
#   id = i
#   start_time = gt[gt.id==id]['start'].item()
#   end_time = gt[gt.id==id]['end'].item()
#   filling_type = gt[gt.id==id]['filling_type'].item()
#   sample_rate, signal = scipy.io.wavfile.read(path)
#   ap = AudioProcessing(sample_rate,signal,nfilt=save_size)
#   mfcc = ap.calc_MFCC()
#   raw_frames = ap.cal_frames()
#   mfcc_length=mfcc.shape[0]

#   if mfcc_length < save_size:
#     print("file {} is too short".format(id))
#   else:
#     f_step=int(mfcc.shape[1]*ratio_step)
#     f_length=mfcc.shape[1]
#     save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) / f_step))

#     for i in range(save_mfcc_num):
#       count += 1
#       tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
#       if start_time == -1:
#           pouring_or_shaking_list.append(0)
#       elif start_time/ap.signal_length_t*mfcc_length<i*f_step+f_length*0.75 and end_time/ap.signal_length_t*mfcc_length>i*f_step+f_length*0.25:
#           pouring_or_shaking_list.append(1) 
#       else:
#           pouring_or_shaking_list.append(0)
      
#       filling_type_list.append(filling_type)
#       file_idx_list.append(id)

#       np.save(os.path.join(mfcc_path, "{0:06d}".format(count)), tmp_mfcc)
#   pbar.update()


# np.save(os.path.join(base_path, 'audios', 'pouring_or_shaking'), np.array(pouring_or_shaking_list) )
# np.save(os.path.join(base_path, 'audios', 'filling_type'), np.array(filling_type_list))


100%|██████████| 684/684 [09:43<00:00,  1.04s/it]

In [8]:
filling_type = np.load(os.path.join(base_path, 'audios', 'filling_type.npy'))
pouring_or_shaking = np.load(os.path.join(base_path,  'audios', 'pouring_or_shaking.npy'))

label = filling_type * pouring_or_shaking

In [22]:
class audioDataSet(Dataset):
  def __init__(self,root_pth,test=False,transform = None):
    print("Dataset initializing...")
    class_num=4
    self.audio_pth = os.path.join(root_pth, 'audios', 'mfcc')
    filling_type = np.load(os.path.join(root_pth, 'audios', 'filling_type.npy'))
    pouring_or_shaking = np.load(os.path.join(root_pth,  'audios', 'pouring_or_shaking.npy'))
    self.label = filling_type * pouring_or_shaking
    self.is_test=test
    self.each_class_size = []
    for i in range(class_num):
        self.each_class_size.append(np.count_nonzero(self.label==i))
    mx=0
    mn=1000
    for idx in tqdm(range(self.label.shape[0])):
        data=np.load(os.path.join(self.audio_pth, "{0:06d}".format(idx+1) + '.npy'), allow_pickle=True)
        tmp_max=np.max(data)
        tmp_min=np.min(data)
        if mx<tmp_max:
            mx=tmp_max
        if mn>tmp_min:
            mn=tmp_min
    self.mn=mn
    self.mx=mx
  def __len__(self):
    return self.label.shape[0]
  def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        lbl = -1

        if self.is_test is False:
            lbl = self.label[idx]
        data=np.load(os.path.join(self.audio_pth, "{0:06d}".format(idx+1) + '.npy'), allow_pickle=True)
        data= (data-self.mn)/(self.mx-self.mn)
        data=data.transpose(2,0,1)
        data=torch.from_numpy(data.astype(np.float32))
        return data , lbl
            
  def get_each_class_size(self):
    return np.array(self.each_class_size)

In [ ]:
from torchinfo import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model = Net(8, 4).to(device)
summary(model, input_size=(32, 8, 64, 64))

In [77]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv01 = nn.Conv2d(8, 32, 3,padding=1)#64
        self.conv02 = nn.Conv2d(32, 32, 3,padding=1)#64
        self.bn1=nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)#32

        self.conv03 = nn.Conv2d(32, 64, 3,padding=1)#32
        self.conv04 = nn.Conv2d(64, 64, 3,padding=1)#32
        self.bn2=nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)#16

        self.conv05 = nn.Conv2d(64, 128, 3,padding=1)#16
        self.conv06 = nn.Conv2d(128, 128, 3,padding=1)#16
        self.conv07 = nn.Conv2d(128, 128, 3,padding=1)#16
        self.bn3=nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)#8

        self.conv08 = nn.Conv2d(128, 256, 3,padding=1)#8
        self.conv09 = nn.Conv2d(256, 256, 3,padding=1)#8
        self.conv10 = nn.Conv2d(256, 256, 3,padding=1)#8
        self.bn4=nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)#4

        self.conv11 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.conv12 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.conv13 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.bn5=nn.BatchNorm2d(256)
        self.pool5 = nn.MaxPool2d(2, 2)#2

       
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 4)

        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=1)



    def forward(self, x):
        x = F.relu(self.conv01(x))
        x = F.relu(self.conv02(x))
        x = self.pool1(self.bn1(x))

        x = F.relu(self.conv03(x))
        x = F.relu(self.conv04(x))
        x = self.pool2(self.bn2(x))

        x = F.relu(self.conv05(x))
        x = F.relu(self.conv06(x))
        x = F.relu(self.conv07(x))
        x = self.pool3(self.bn3(x))

        x = F.relu(self.conv08(x))
        x = F.relu(self.conv09(x))
        x = F.relu(self.conv10(x))
        x = self.pool4(self.bn4(x))

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.pool5(self.bn5(x))


        x = x.view(-1, 256 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return self.softmax(x)   

In [ ]:
mydataset = audioDataSet(base_path)

In [92]:
def train(model, train_loader, optimizer, criterion = nn.CrossEntropyLoss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)

    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    loss_train += loss.item() / num_train
    _, preds=torch.max(outputs,1)
    correct_train+=torch.sum(preds==target).item()
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.CrossEntropyLoss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / num_val
      _, preds=torch.max(outputs,1)
      correct_test+=torch.sum(preds==target).item()
  
  return loss_test, correct_test


In [98]:
bs = 100
train_split = 0.8
lr = 1e-5
epochs = 200
n_samples = len(mydataset)
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True)
for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("Epoch {}/{} train loss:{:.4f} train acc:{:.2f}% ".format(epoch+1,epochs, loss_train, 100 * correct_train/num_train))
  print("Epoch {}/{} val loss:{:.4f} val acc:{:.2f}% ".format(epoch+1,epochs, loss_val, 100 * correct_val/num_val))

  if loss_val < best_loss:
    best_loss = loss_val
    torch.save(model, os.path.join(base_path, 'audios', "best_loss.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model, os.path.join(base_path, 'audios', "best_val.pth"))
  




Epoch 1/200 train loss:1.0360 train acc:73.44% 
Epoch 1/200 val loss:0.9534 val acc:79.19% 
Epoch 2/200 train loss:0.9431 train acc:80.36% 
Epoch 2/200 val loss:0.9258 val acc:81.64% 
Epoch 3/200 train loss:0.9241 train acc:81.98% 
Epoch 3/200 val loss:0.9357 val acc:80.56% 
Epoch 4/200 train loss:0.9132 train acc:82.80% 
Epoch 4/200 val loss:0.9041 val acc:84.02% 
Epoch 5/200 train loss:0.8858 train acc:86.28% 
Epoch 5/200 val loss:1.0089 val acc:72.62% 
Epoch 6/200 train loss:0.8672 train acc:87.78% 
Epoch 6/200 val loss:0.8978 val acc:84.50% 
Epoch 7/200 train loss:0.8605 train acc:88.34% 
Epoch 7/200 val loss:0.9934 val acc:74.57% 
Epoch 8/200 train loss:0.8526 train acc:89.10% 
Epoch 8/200 val loss:0.8889 val acc:85.34% 
Epoch 9/200 train loss:0.8470 train acc:89.68% 
Epoch 9/200 val loss:0.8650 val acc:87.79% 
Epoch 10/200 train loss:0.8426 train acc:90.15% 
Epoch 10/200 val loss:0.8547 val acc:88.86% 
Epoch 11/200 train loss:0.8390 train acc:90.55% 
Epoch 11/200 val loss:0.8496 

In [105]:
from mobile import MobileNetV3_Large

bs = 100
train_split = 0.8
lr = 1e-3
epochs = 200
n_samples = len(mydataset)
model = MobileNetV3_Large().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True)
for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))

  if loss_val < best_loss:
    best_loss = loss_val
    torch.save(model, os.path.join(base_path, 'audios', "bl-mobile.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model, os.path.join(base_path, 'audios', "bv-mobile.pth"))
  




1/200 train loss:0.5775 train acc:79.35% val loss:0.5985 val acc:79.21%
2/200 train loss:0.3572 train acc:87.87% val loss:0.3697 val acc:86.89%
3/200 train loss:0.2895 train acc:89.89% val loss:0.3728 val acc:87.68%
4/200 train loss:0.2644 train acc:90.48% val loss:0.3346 val acc:88.31%
5/200 train loss:0.2458 train acc:91.26% val loss:0.5997 val acc:73.06%
6/200 train loss:0.2228 train acc:91.91% val loss:0.4331 val acc:88.59%
7/200 train loss:0.2042 train acc:92.81% val loss:0.2907 val acc:89.39%
8/200 train loss:0.1795 train acc:93.65% val loss:0.6188 val acc:78.47%
9/200 train loss:0.1554 train acc:94.29% val loss:0.2454 val acc:91.56%
10/200 train loss:0.1468 train acc:94.63% val loss:0.5624 val acc:80.92%
11/200 train loss:0.1265 train acc:95.43% val loss:0.3431 val acc:89.66%
12/200 train loss:0.1140 train acc:95.92% val loss:0.4471 val acc:88.10%
13/200 train loss:0.1017 train acc:96.37% val loss:0.3115 val acc:91.62%
14/200 train loss:0.0938 train acc:96.56% val loss:0.3609 va

In [ ]:
model_pretrained = torch.load(os.path.join(base_path, 'audios', "bv-mobile.pth"))
model_pretrained.to(device)
model_pretrained.eval()

voting_dir = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios'

mfcc_MAX_VALUE=194.19187653405487
mfcc_MIN_VALUE=-313.07119549054045

t2_MAX_VALUE = 57.464638
t2_MIN_VALUE = -1.1948369
save_size=64

start = time.time()

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
save_data = {}
data_num = 0
for i, path in enumerate(audio_paths):
  count_pred = [0] * 4
  pred_list = []
  sample_rate, signal = scipy.io.wavfile.read(path)
  ap = AudioProcessing(sample_rate,signal)
  mfcc = ap.calc_MFCC()
  mfcc_length=mfcc.shape[0]
  f_step=int(mfcc.shape[1]*0.25)
  f_length=mfcc.shape[1]
  save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) /f_step))
  for i in range(save_mfcc_num):
      tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
      tmp_mfcc= (tmp_mfcc-mfcc_MIN_VALUE)/(mfcc_MAX_VALUE-mfcc_MIN_VALUE)
      tmp_mfcc=tmp_mfcc.transpose(2,0,1)
      audio=torch.from_numpy(tmp_mfcc.astype(np.float32))
      audio=torch.unsqueeze(audio, 0)
      audio = audio.to(device)

      with torch.no_grad():
        pred_T2 = model_pretrained.forward(audio)
        _,pred_T2=torch.max(pred_T2,1)
        count_pred[pred_T2.item()]+=1
        pred_list.append(pred_T2.item())
  if  count_pred[1]>5 or count_pred[2]>5 or count_pred[3]>5:
    final_pred_T2=count_pred[1:4].index(max(count_pred[1:4]))+1
  else:
    final_pred_T2=0
  
  file_name = path.split(os.path.sep)[-1].replace('.wav', '')
  #print("sequence:{}, frequency:{}".format(file_name, count_pred))
  to_save_data = {"data_num":data_num,
                  "file":file_name,
                  "count_pred":count_pred,
                  "final_pred":final_pred_T2,
                  'pred':pred_list}
  save_data["{}".format(file_name)] = to_save_data
  data_num+=1

with open (os.path.join(voting_dir, "voting.json"), 'w') as f:
  json.dump(save_data, f, indent=2, ensure_ascii=False)
  elapsed_time = time.time() - start
  print("elapsed_time:{}".format(elapsed_time) + "sec")



In [121]:
f = open(os.path.join(voting_dir, "voting.json"))
vote_js = json.load(f)

vote = pd.DataFrame(vote_js).T
vote.head()

,data_num,file,count_pred,final_pred,pred
000000,0,000000,"[19, 0, 19, 0]",2,"[0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
000001,1,000001,"[13, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
000002,2,000002,"[56, 0, 0, 21]",3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
000003,3,000003,"[16, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
000004,4,000004,"[7, 9, 0, 0]",1,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]"


In [136]:
gt = pd.read_csv('/content/drive/MyDrive/COSRMAL_CHALLENGE/train.csv')
acc = np.sum(gt['filling_type'].to_numpy() == vote['final_pred'].to_numpy()) / len(gt['filling_type'])
print('Acc: {:.2f}%'.format(100 * acc))

Acc: 100.00%


(684,)